В файл вынесены функции помогающие парсить вакансии с сайта hh.ru

In [1]:
import requests
import json
from lxml import html
from bs4 import BeautifulSoup as bs
from time import sleep
from datetime import datetime
import pandas as pd

In [2]:
#функция запичи страницы с вакансия с сайта hh.ru

def page_for_parsing(search_text, num_page):

    url = 'https://hh.ru/search/vacancy'

    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36',
            }

    params = {'text': search_text, 'page': num_page}

    return requests.get(url=url, headers=headers, params=params)

In [3]:
#функция определения страниц в поиске

def max_find_page(button_whith_number_find_pages):
    return max([int(el) for el in button_whith_number_find_pages if el.isdigit()])           

In [4]:
#функция определения минимальной и максимальной зарплаты

def min_max_salary(salary_data):
    
    currency_str=salary_data
    salary_data.replace(" ", "")
    min_salary, max_salary = '', ''
    
    if salary_data[0].isdigit()or salary_data[0]=='о':
        j = 1
    elif salary_data[0]=='д':
        j = 2
    
    for el in salary_data:
        if j==1 and el.isdigit():
            min_salary += el
        elif el=='–':
            j = 2
        if j==2 and el.isdigit():
            max_salary += el
            
    if min_salary == '':
        min_salary = None
    else:
        min_salary = float(min_salary)
        
    if max_salary == '':
        max_salary = None
    else:
        max_salary = float(max_salary)
        
    currency = currency_str[currency_str.rfind(" ")+1:]
    
    return [min_salary, max_salary, currency]
            

In [5]:
#функция формирования данных по вакансии

def vacancy_data(information, search_text):
    vacancy_name = information.find('a', {'class':'serp-item__title'}).text
    vacancy_url = information.find('a', {'class':'serp-item__title'}, href=True)['href']
    try:
        salary_data = information.find('span', {'class':'bloko-header-section-3'}).text
    except:
        salary_data = None
    try:
        employer = information.find('a', {'class':'bloko-link bloko-link_kind-tertiary'}).text
    except:
        employer = None
    try:
        employer_url = 'https://hh.ru'+information.find('a', {'class':'bloko-link bloko-link_kind-tertiary'}, href=True)['href']
    except:
        employer_url = None
    try:
        adress = information.find('div', {'data-qa':'vacancy-serp__vacancy-address'}).text
    except:
        employer_url = None
    try:
        short_description_of_vacancy = (information.find('div', {'data-qa':'vacancy-serp__vacancy_snippet_responsibility'}).text +' '
                                    +information.find('div', {'data-qa':'vacancy-serp__vacancy_snippet_requirement'}).text)
    except:
        short_description_of_vacancy = None
        
    if salary_data==None:
        salary_list = [None, None, None]
    else:
        salary_list = min_max_salary(salary_data)
    
    vacancy_dict = {
        'search_name': search_text,
        'date_search': datetime.now(),
        'hh_site': True,
        'vacancy_name':vacancy_name,
        'vacancy_url':vacancy_url, 
        'min_salary': salary_list[0],
        'max_salary': salary_list[1],
        'currency': salary_list[2],
        'employer': employer,
        'employer_url':employer_url,
        'adress': adress,
        'short_description_of_vacancy': short_description_of_vacancy}
    
    return vacancy_dict

In [6]:
def hh_search_parsing_db_writing(search_text, db):

    first_search_page = page_for_parsing(search_text, "0")

    first_soup = bs(first_search_page.content, 'html.parser')
    first_dom = html.fromstring(first_search_page.text)
    
    button_whith_number_find_pages = first_dom.xpath("//a[@class='bloko-button']/span/text()")
    max_page = max_find_page(button_whith_number_find_pages)
    
    vacancy_infromation_soup = first_soup.find_all('div', {'class':'serp-item'})
    

    for el in vacancy_infromation_soup:
        db.hh.insert_one(vacancy_data(el, search_text))
        
    

    for num_page in range(1, max_page):
        sleep(0.5)
        search_page = page_for_parsing(search_text, num_page)
        soup = bs(search_page.content, 'html.parser')
        vacancy_infromation_soup = soup.find_all('div', {'class':'serp-item'})
        for el in vacancy_infromation_soup:
             db.hh.insert_one(vacancy_data(el, search_text))
            